In [ ]:
# Setting up required libraries
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [ ]:
# Connect to website to be scraped, and get all html
url1='https://results.virginmoneylondonmarathon.com/2020/?page='
url2='&event=ALL&num_results=1000&pid=search&pidp=results_nav&search%5Bsex%5D='
url3='&search%5Bage_class%5D=%25&search%5Bnation%5D=%25&search_sort=name'

#Get results for men, 878 pages of results <-There is no search option for other gender/sex
mens_results = pd.DataFrame()
for i in range(22):
    sex='M'
    website_url=requests.get(url1+str(i)+url2+sex+url3).text
    soup = BeautifulSoup(website_url,'lxml')

    fields = soup.find(class_='section-main')

    my_table = []
    for row in fields.findAll(class_='list-group-item'):
        row_data = []
        for cell in row.findAll(class_='list-field'):
            row_data.append(cell.text)

        if(len(row_data) > 0):
            data_item = {"Place (Overall)": row_data[0],
                         "Place (Gender)": row_data[1],
                         "Place (Category)": row_data[2],
                         "Name": row_data[3],
                         "Sex": sex,
                         "Club": row_data[4],
                         "Running Number": row_data[5],
                         "Category": row_data[6],
                         "Finish": row_data[7],
            }
            my_table.append(data_item)

    df = pd.DataFrame(my_table).iloc[1:]
    
    mens_results = mens_results.append(df)

In [ ]:
#Get results for women
womens_results = pd.DataFrame()

for i in range(22):
    sex='F'
    website_url=requests.get(url1+str(i)+url2+sex+url3).text
    soup = BeautifulSoup(website_url,'lxml')

    fields = soup.find(class_='section-main')

    my_table = []
    for row in fields.findAll(class_='list-group-item'):
        row_data = []
        for cell in row.findAll(class_='list-field'):
            row_data.append(cell.text)

        if(len(row_data) > 0):
            data_item = {"Place (Overall)": row_data[0],
                         "Place (Gender)": row_data[1],
                         "Place (Category)": row_data[2],
                         "Name": row_data[3],
                         "Sex": sex,
                         "Club": row_data[4],
                         "Running Number": row_data[5],
                         "Category": row_data[6],
                         "Finish": row_data[7],
            }
            my_table.append(data_item)

    df = pd.DataFrame(my_table).iloc[1:]
    
    womens_results = womens_results.append(df)

In [19]:
# Concatenate results
results = pd.concat([mens_results, womens_results])

In [20]:
#Some quick data cleaning
#Remove leftover titles
results['Club'] = results['Club'].str.replace("Club", "", regex=False)
results['Running Number'] = results['Running Number'].str.replace("Running Number", "", regex=False)
results['Category'] = results['Category'].str.replace("Category", "", regex=False)
results['Finish'] = results['Finish'].str.replace("Finish", "", regex=False)

#Extract country groups, like (USA), from Name group
results['Country'] = results['Name'].str.extract(r'(\([A-Z]{3,}\))')
#Remove brackets in country
results['Country'] = results['Country'].str.replace(r'\(|\)', "")
#Remove country group from name column
results['Name'] = results['Name'].str.replace(r'(\([A-Z]{3}\))', "")

#Split first/lastname into new columns
LastFirst = results['Name'].str.split(pat=",", n=1, expand=True) 
results['FirstName'], results['LastName'] = LastFirst[1], LastFirst[0]
#Remove comma from Name column, so that this can be saved as a CSV ----- Must happen after splitting Name into two cols!!
results['Name'] = results['Name'].str.replace(r'(\,)', "")
#Replace non-standard '–' with NaN for missing vals
results = results.replace('–', np.nan)

In [21]:
results

,Place (Overall),Place (Gender),Place (Category),Name,Sex,Club,Running Number,Category,Finish,Country,FirstName,LastName
1,NaN,NaN,NaN,Ashdjian Thérèse,F,NaN,12899,50-54,NaN,GBR,Thérèse,Ashdjian
2,10809,3248,382,Ashdown Katharine,F,NaN,36539,50-54,04:34:45,GBR,Katharine,Ashdown
3,NaN,NaN,NaN,Ashdown-tams Luke,F,NaN,36422,18-39,NaN,GBR,Luke,Ashdown-tams
4,9607,2766,584,Ashe Finola,F,NaN,17279,45-49,04:27:08,GBR,Finola,Ashe
5,25293,10875,1403,Asher Joanne,F,NaN,15943,50-54,06:34:44,GBR,Joanne,Asher
...,...,...,...,...,...,...,...,...,...,...,...,...
996,NaN,NaN,NaN,Kozak Lauren,F,NaN,42820,18-39,NaN,USA,Lauren,Kozak
997,3068,2533,1041,Kozak Vitalii,F,NaN,13199,18-39,03:38:43,GBR,Vitalii,Kozak
998,NaN,NaN,NaN,Koziol Andy,F,NaN,13924,45-49,NaN,CAN,Andy,Koziol
999,25272,14413,5151,Kozior Konrad,F,NaN,41829,18-39,06:34:31,GBR,Konrad,Kozior


In [ ]:
# And quickly save them in a csv
results.to_csv(r'C:\Users\michael.walshe\Documents\Python Projects\Scraping_London_Marathon_Project\London_2020.csv', index=False, header=True)